In [1]:
!pip install rasterio geopandas pandas numpy matplotlib

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/19.2 MB ? eta -:--:--
    --------------------------------------- 0.3/19.2 MB ? eta -:--:--
   - -------------------------------------- 0.8/19.2 MB 2.1 MB/s eta 0:00:09
   -- ------------------------------------- 1.0/19.2 MB 1.9 MB/s eta 0:00:10
   --- ------------------------------------ 1.6/19.2 MB 2.0 MB/s eta 0:00:09
   ---- ----------------------------------- 2.1/19.2 MB 2.1 MB/s eta 0:00:09
   ---- ----------------------------------- 2.4/19.2 MB 2.1 MB/s eta 0:00:09
   ----- ---------------------------------- 2.6/19.2 MB 2.0 MB/s eta 0:00:09
   ------ --------------------------------- 3.1/19.2 MB 1.8 MB/s eta 0:00:09
   ------- -------------------------------- 3.4/19.2 MB 1.8 MB/s eta 0:00:09
   ------- -------------------------------- 3.4/19.2 MB 1.8 MB/s eta 0:00:09
   ------- -------------------------------- 3.7/19.2 MB 1.6 MB/s eta 0:00:10
   ------- -

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 25.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
!pip install earthengine-api

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 25.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
!pip install ultralytics

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/1.0 MB ? eta -:--:--
   -------------------- ------------------- 0.5/1.0 MB 2.4 MB/s eta 0:00:01
   ---------------------------------------- 1.0/1.0 MB 2.8 MB/s eta 0:00:00


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 25.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
!pip install folium

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 25.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import ee
import time

# --- Initialize the Earth Engine API ---
try:
    ee.Initialize()
    print("Google Earth Engine API initialized successfully.")
except ee.EEException:
    print("Authentication failed. Please run 'earthengine authenticate' in your terminal.")
    exit()

# --- Configuration ---
# Define your Area of Interest (AOI).
# For this example, we'll use a sample rectangle over a river area.
# Replace this with your own ee.Geometry or feature collection.
AOI = ee.Geometry.Rectangle([88.20, 22.00, 88.30, 22.05]) # Example: Part of Hooghly River, India

# Define time periods for 'before' and 'after' analysis
BEFORE_START_DATE = '2022-01-01'
BEFORE_END_DATE = '2022-03-31'
AFTER_START_DATE = '2023-01-01'
AFTER_END_DATE = '2023-03-31'

# --- Sentinel-2 (Optical) Data Processing ---
print("Processing Sentinel-2 Optical Data...")

def mask_s2_clouds(image):
    """Masks clouds in a Sentinel-2 image using the S2 cloud probability asset."""
    qa = image.select('QA60')
    # Bits 10 and 11 are clouds and cirrus, respectively.
    cloud_bit_mask = 1 << 10
    cirrus_bit_mask = 1 << 11
    # Both flags should be set to zero, indicating clear conditions.
    mask = qa.bitwiseAnd(cloud_bit_mask).eq(0).And(qa.bitwiseAnd(cirrus_bit_mask).eq(0))
    return image.updateMask(mask).divide(10000) # Scale to surface reflectance values

# Get 'before' and 'after' Sentinel-2 collections
s2_before_collection = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
   .filterDate(BEFORE_START_DATE, BEFORE_END_DATE) \
   .filter(ee.Filter.bounds(AOI)) \
   .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)) \
   .map(mask_s2_clouds)

s2_after_collection = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
   .filterDate(AFTER_START_DATE, AFTER_END_DATE) \
   .filter(ee.Filter.bounds(AOI)) \
   .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)) \
   .map(mask_s2_clouds)

# Create median composites to get a single, clear image for each period [4]
s2_before_image = s2_before_collection.median().clip(AOI)
s2_after_image = s2_after_collection.median().clip(AOI)

# --- Sentinel-1 (SAR) Data Processing ---
print("Processing Sentinel-1 SAR Data...")

# Get 'before' and 'after' Sentinel-1 collections [1]
s1_before_collection = ee.ImageCollection('COPERNICUS/S1_GRD') \
   .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV')) \
   .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH')) \
   .filter(ee.Filter.eq('instrumentMode', 'IW')) \
   .filter(ee.Filter.bounds(AOI)) \
   .filterDate(BEFORE_START_DATE, BEFORE_END_DATE)

s1_after_collection = ee.ImageCollection('COPERNICUS/S1_GRD') \
   .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV')) \
   .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH')) \
   .filter(ee.Filter.eq('instrumentMode', 'IW')) \
   .filter(ee.Filter.bounds(AOI)) \
   .filterDate(AFTER_START_DATE, AFTER_END_DATE)

# Create median composites
s1_before_image = s1_before_collection.median().clip(AOI)
s1_after_image = s1_after_collection.median().clip(AOI)

# --- Exporting to Google Drive ---
print("Starting export tasks to Google Drive...")

# Export Sentinel-2 images
task_s2_before = ee.batch.Export.image.toDrive(
    image=s2_before_image.select(), # Red, Green, Blue, NIR
    description='S2_Before_Composite',
    folder='SandMiningDetection_Exports',
    fileNamePrefix='s2_before',
    region=AOI,
    scale=10,
    crs='EPSG:4326'
)
task_s2_before.start()

task_s2_after = ee.batch.Export.image.toDrive(
    image=s2_after_image.select(),
    description='S2_After_Composite',
    folder='SandMiningDetection_Exports',
    fileNamePrefix='s2_after',
    region=AOI,
    scale=10,
    crs='EPSG:4326'
)
task_s2_after.start()

# Export Sentinel-1 images
task_s1_before = ee.batch.Export.image.toDrive(
    image=s1_before_image.select(['VV', 'VH']),
    description='S1_Before_Composite',
    folder='SandMiningDetection_Exports',
    fileNamePrefix='s1_before',
    region=AOI,
    scale=10,
    crs='EPSG:4326'
)
task_s1_before.start()

task_s1_after = ee.batch.Export.image.toDrive(
    image=s1_after_image.select(['VV', 'VH']),
    description='S1_After_Composite',
    folder='SandMiningDetection_Exports',
    fileNamePrefix='s1_after',
    region=AOI,
    scale=10,
    crs='EPSG:4326'
)
task_s1_after.start()

# --- Monitor Task Status ---
while task_s2_after.active() or task_s1_after.active():
    print(f"Monitoring export tasks... S2_After: {task_s2_after.status()['state']}, S1_After: {task_s1_after.status()['state']}")
    time.sleep(60)

print("All export tasks have been submitted. Check your Google Drive 'SandMiningDetection_Exports' folder.")


Authentication failed. Please run 'earthengine authenticate' in your terminal.


EEException: Caller does not have required permission to use project 517222506229. Grant the caller the roles/serviceusage.serviceUsageConsumer role, or a custom role with the serviceusage.services.use permission, by visiting https://console.developers.google.com/iam-admin/iam/project?project=517222506229 and then retry. Propagation of the new permission may take a few minutes.

In [1]:
import ee

try:
    # Initialize the library.
    ee.Initialize()
    print("Successfully connected to Google Earth Engine!")

    # Run a simple test: get information about an image.
    image = ee.Image('srtm90_v4')
    print("SRTM image info:", image.getInfo())

except Exception as e:
    print("Could not connect to Earth Engine.", e)

Could not connect to Earth Engine. ee.Initialize: no project found. Call with project= or see http://goo.gle/ee-auth.


In [2]:
!earthengine authenticate

Authenticate: Credentials already exist.  Use --force to refresh.
